# Решение #5 - Частотный анализ

## Содержание
* [Описание решения](#description)
* [Подготовительные действия](#preprocess)
* [Решение](#solution)
* [Результаты](#result)

---

## Описание решения <a id='description'></a>

Решение основано на исправлении всех опечаток в датасете за счет подбора ближайшего известного корректного слова, а также проверке смысловых паттернов, таких как проверка местоимений и отрицаний. На основе датасета строится частотный словарь слов, на основе которого строится BK-дерево, которое используется для нахождения орфографически верных написаний.

Решение использует следующие структуры и алгоритмы:
* Расстояние Левенштейна
* BK-деревья
* Офография на основе частоты появления слова
* Проверка набора слов
* Проверка наборов местоимений
* Проверка отрицания


---

## Подготовительные действия <a id='preprocess'></a>
### Импортирование библиотек и загрузка датасета

In [1]:
from src import *
from tqdm.auto import tqdm
from ipywidgets import FloatProgress
import pandas as pd
import json

In [2]:
base_data = load_dataset.load('../data/sample.json')

In [3]:
base_data

['Ты нашёл их или нет?',
 'Почему она так со мной поступает?',
 'Никто туда больше не ходит.',
 'У него с собой не было тогда денег.',
 'Почему они с нами так поступают?',
 'Он всю ночь стонал от сильной боли.',
 'Я больше не хочу с тобой играть.',
 'Тому было тогда всего тринадцать лет.',
 'Что сделал Том с деньгами?',
 'Том меня сейчас хочет видеть?',
 'Он даже меня не замечает.',
 'Тебе это всё нравится?',
 'У него тогда не было с собой денег.',
 'Я его больше не увижу.',
 'Почему она так с ней поступает?',
 'Я хотел бы учиться в Бостоне.',
 'Том и Мэри объявили сегодня о своей пбмолвке.',
 'Том этим сейчас занимается.',
 'Я не могу больше ждать.',
 'Мост очень длинный и высокий.',
 'Том был просто не готов.',
 'Пусть свиньи это едят.',
 'Я ничего не хочу делать.',
 'Ты хотел мне рассказать о свободе?',
 'Что пел Джон на сцене?',
 'Я написал влера письмо.',
 'Тому это тоже не нравится.',
 'Сколько сейчас времени в Париже?',
 'Мы не были готовы.',
 'Они их только что нашли.',
 'Тому 

### Предобработка строк

In [4]:
data = [load_dataset.preprocess_text(text) for text in base_data] # Убираем знаки препинания и лишние пробелы

In [5]:
data

['ты нашел их или нет',
 'почему она так со мной поступает',
 'никто туда больше не ходит',
 'у него с собой не было тогда денег',
 'почему они с нами так поступают',
 'он всю ночь стонал от сильной боли',
 'я больше не хочу с тобой играть',
 'тому было тогда всего тринадцать лет',
 'что сделал том с деньгами',
 'том меня сейчас хочет видеть',
 'он даже меня не замечает',
 'тебе это все нравится',
 'у него тогда не было с собой денег',
 'я его больше не увижу',
 'почему она так с ней поступает',
 'я хотел бы учиться в бостоне',
 'том и мэри объявили сегодня о своей пбмолвке',
 'том этим сейчас занимается',
 'я не могу больше ждать',
 'мост очень длинный и высокий',
 'том был просто не готов',
 'пусть свиньи это едят',
 'я ничего не хочу делать',
 'ты хотел мне рассказать о свободе',
 'что пел джон на сцене',
 'я написал влера письмо',
 'тому это тоже не нравится',
 'сколько сейчас времени в париже',
 'мы не были готовы',
 'они их только что нашли',
 'тому было больше не к кому обратить

---

## Решение <a id='solution'></a>
### Создание орфографического словаря

In [6]:
# Находим уникальные слова в датасете и их частоту
actual_word_set = dict()
for line in data:
    words = line.split()
    for word in words:
        old = actual_word_set.get(word, 0)
        actual_word_set[word] = old + 1

In [7]:
# Загружаем орфографический словарь
vocabulary_source = "../data/russian-utf8.json"
vocabulary = pd.read_json(vocabulary_source)[0].tolist()

In [8]:
vocabulary

['автобусе',
 'америке',
 'апельсины',
 'банан',
 'банкомат',
 'без',
 'безопасности',
 'белое',
 'беру',
 'библиотеки',
 'боли',
 'больше',
 'бостоне',
 'будем',
 'будет',
 'будешь',
 'буду',
 'будут',
 'бы',
 'был',
 'была',
 'были',
 'было',
 'быстро',
 'быть',
 'в',
 'важно',
 'вам',
 'вас',
 'ввести',
 'вернется',
 'весь',
 'вечером',
 'видел',
 'видеть',
 'вино',
 'вместе',
 'внутри',
 'во',
 'воды',
 'возится',
 'возьми',
 'вот',
 'времени',
 'время',
 'все',
 'всегда',
 'всего',
 'всем',
 'встретил',
 'всю',
 'вчера',
 'вы',
 'вынужден',
 'выпил',
 'выражение',
 'высадить',
 'высокая',
 'высокий',
 'высоким',
 'выучить',
 'газету',
 'где',
 'говорит',
 'говорить',
 'говорю',
 'говорят',
 'года',
 'году',
 'голос',
 'гора',
 'горах',
 'гордился',
 'готов',
 'готова',
 'готовы',
 'границу',
 'даже',
 'дал',
 'девочкой',
 'действует',
 'делать',
 'деле',
 'денег',
 'день',
 'деньгами',
 'деньги',
 'десяти',
 'детстве',
 'джон',
 'длинный',
 'для',
 'дня',
 'до',
 'доверяет',
 'дов

In [9]:
# Загружаем имена
names_source = "../data/names.json"
names = pd.read_json(names_source)[0].tolist()

In [10]:
names

['александр',
 'николай',
 'иван',
 'сергей',
 'владимир',
 'михаил',
 'василий',
 'елена',
 'татьяна',
 'алексей',
 'андрей',
 'наталья',
 'ольга',
 'дмитрий',
 'мария',
 'виктор',
 'анна',
 'ирина',
 'светлана',
 'валентина',
 'юрий',
 'петр',
 'людмила',
 'анатолий',
 'галина',
 'павел',
 'игорь',
 'нина',
 'евгений',
 'олег',
 'екатерина',
 'надежда',
 'александра',
 'валерий',
 'вера',
 'виталий',
 'марина',
 'тамара',
 'григорий',
 'федор',
 'любовь',
 'борис',
 'геннадий',
 'юлия',
 'леонид',
 'константин',
 'лидия',
 'вячеслав',
 'антонина',
 'анастасия',
 'оксана',
 'лариса',
 'зинаида',
 'георгий',
 'денис',
 'валентин',
 'клавдия',
 'максим',
 'евгения',
 'вадим',
 'алла',
 'раиса',
 'зоя',
 'роман',
 'илья',
 'руслан',
 'инна',
 'наталия',
 'евдокия',
 'виктория',
 'степан',
 'семен',
 'антон',
 'лилия',
 'эдуард',
 'станислав',
 'жанна',
 'яков',
 'елизавета',
 'аркадий',
 'владислав',
 'маргарита',
 'артем',
 'кирилл',
 'егор',
 'артур',
 'таисия',
 'пелагея',
 'эльвира',

In [11]:
vocabulary = vocabulary + names

In [12]:
# Удаляем слова, которые не задействованы в датасете
vocabulary = [item for item in vocabulary if item in actual_word_set]
encounter_rate = [actual_word_set[word] for word in vocabulary]

In [13]:
# Строим BK-дерево
verification_tree = bk_tree.bk_tree(vocabulary, encounter_rate)
verification_tree.build()

  0%|          | 0/530 [00:00<?, ?it/s]

### Исправление орфографических ошибок

In [14]:
# Функция коррекции орфографических ошибок
def fix_spelling(data, correct_tree):
    for i in tqdm(range(len(data))):
        line = data[i]
        words = line.split()
        for j in range(len(words)):
            words[j] = correct_tree.find_correct(words[j], 2)
        data[i] = ' '.join(words)
    return data

In [15]:
data = fix_spelling(data, verification_tree)

  0%|          | 0/412 [00:00<?, ?it/s]

In [16]:
data

['ты нашел их или нет',
 'почему она так со мной поступает',
 'никто туда больше не ходит',
 'у него с собой не было тогда денег',
 'почему они с нами так поступают',
 'он всю ночь стонал от сильной боли',
 'я больше не хочу с тобой играть',
 'тому было тогда всего тринадцать лет',
 'что сделал том с деньгами',
 'том меня сейчас хочет видеть',
 'он даже меня не замечает',
 'тебе это все нравится',
 'у него тогда не было с собой денег',
 'я его больше не увижу',
 'почему она так с ней поступает',
 'я хотел бы учиться в бостоне',
 'том и мэри объявили сегодня о своей помолвке',
 'том этим сейчас занимается',
 'я не могу больше ждать',
 'мост очень длинный и высокий',
 'том был просто не готов',
 'пусть свиньи это едят',
 'я ничего не хочу делать',
 'ты хотел мне рассказать о свободе',
 'что пел джон на сцене',
 'я написал вчера письмо',
 'тому это тоже не нравится',
 'сколько сейчас времени в париже',
 'мы не были готовы',
 'они их только что нашли',
 'тому было больше не к кому обратить

### Группировка предложений

In [17]:
# Функция группировки строк
def find_groups(data, base_data):
    groups = []                        # Cписок для сохранения групп
    threshold = 0.85                   # Ограничение для алгоритмов верификации рерайтов
    used = [False for _ in range(len(data))]    # Список меток использования строк
    for current_id in tqdm(range(len(data))):
        if (used[current_id]):
            continue
        current_group = []             # Текущая группа
        current_group.append(base_data[current_id])
        used[current_id] = True
        for next_id in range(current_id + 1, len(data)):
            #print("\tNext id:", next_id)
            if (used[next_id]):
                continue
            line1 = data[current_id]
            line2 = data[next_id]
            # Проверка строк с помощью алгоритмов верификации
            if (word_collation.is_rewrite_word_collation(line1, line2, threshold) and
               negatives.check_negative_corresponense(line1, line2) and
               (pronouns.check_pronoun_correspondence(line1, line2) 
                or (hamming_distance_custom.clean_hamming_distance(line1, line2) == 1))):
                current_group.append(base_data[next_id])  # Добавление строки в текущую группу
                used[next_id] = True
        groups.append(current_group)             # Добавление сформированной группы к результирующему списку
    return groups

In [18]:
groups = find_groups(data, base_data)

  0%|          | 0/412 [00:00<?, ?it/s]

In [19]:
# Проверка, что строки не были взяты дважды
assert(sum([len(group) for group in groups]) == len(base_data)) 

---

## Результаты <a id='result'></a>

### Все найденные группы

In [20]:
print('Количество групп = ',len(groups))

Количество групп =  281


In [21]:
groups

[['Ты нашёл их или нет?'],
 ['Почему она так со мной поступает?'],
 ['Никто туда больше не ходит.', 'Никто больше туда не ходит.'],
 ['У него с собой не было тогда денег.',
  'У него тогда не было с собой денег.'],
 ['Почему они с нами так поступают?'],
 ['Он всю ночь стонал от сильной боли.'],
 ['Я больше не хочу с тобой играть.', 'Я не хочу с тобой больше играть.'],
 ['Тому было тогда всего тринадцать лет.'],
 ['Что сделал Том с деньгами?', 'Что сделёл Том с деньгами?'],
 ['Том меня сейчас хочет видеть?', 'Том хочет меня сейчас видеть?'],
 ['Он даже меня не замечает.'],
 ['Тебе это всё нравится?'],
 ['Я его больше не увижу.', 'Я больше его не увижу.'],
 ['Почему она так с ней поступает?'],
 ['Я хотел бы учиться в Бостоне.'],
 ['Том и Мэри объявили сегодня о своей пбмолвке.',
  'Том и Мэри объявили сегодня о своей помолвке.'],
 ['Том этим сейчас занимается.'],
 ['Я не могу больше ждать.', 'Я больше не могу ждать.'],
 ['Мост очень длинный и высокий.', 'Мост очень длинный и очень высоки

### Строки, которые имеют более 1 рерайта

In [22]:
groups3 = [group for group in groups if len(group) > 2]

In [23]:
print('Количество групп = ',len(groups3))

Количество групп =  1


In [24]:
groups3

[['Том не хочет никого видеть.',
  'Том не хочет сегодня никого видеть.',
  'Том сегодня не хочет никого видеть.']]

### Строки, которые имеют ровно 1 рерайт

In [25]:
groups2 = [group for group in groups if len(group) == 2]

In [26]:
print('Количество групп = ',len(groups2))

Количество групп =  129


In [27]:
groups2

[['Никто туда больше не ходит.', 'Никто больше туда не ходит.'],
 ['У него с собой не было тогда денег.',
  'У него тогда не было с собой денег.'],
 ['Я больше не хочу с тобой играть.', 'Я не хочу с тобой больше играть.'],
 ['Что сделал Том с деньгами?', 'Что сделёл Том с деньгами?'],
 ['Том меня сейчас хочет видеть?', 'Том хочет меня сейчас видеть?'],
 ['Я его больше не увижу.', 'Я больше его не увижу.'],
 ['Том и Мэри объявили сегодня о своей пбмолвке.',
  'Том и Мэри объявили сегодня о своей помолвке.'],
 ['Я не могу больше ждать.', 'Я больше не могу ждать.'],
 ['Мост очень длинный и высокий.', 'Мост очень длинный и очень высокий.'],
 ['Пусть свиньи это едят.', 'Пусть это свиньи едят.'],
 ['Ты хотел мне рассказать о свободе?', 'Ты хотел рассказать мне о свободе?'],
 ['Что пел Джон на сцене?', 'Что Джон пел на сцене?'],
 ['Я написал влера письмо.', 'Я написал вчера письмо.'],
 ['Мы не были готовы.', 'Мы были не готовы.'],
 ['Они их только что нашли.', 'Они только что их нашли.'],
 ['

### Строки, которые не имеют рерайта

In [28]:
groups1 = [group for group in groups if len(group) == 1]

In [29]:
print('Количество групп = ',len(groups1))

Количество групп =  151


In [30]:
groups1

[['Ты нашёл их или нет?'],
 ['Почему она так со мной поступает?'],
 ['Почему они с нами так поступают?'],
 ['Он всю ночь стонал от сильной боли.'],
 ['Тому было тогда всего тринадцать лет.'],
 ['Он даже меня не замечает.'],
 ['Тебе это всё нравится?'],
 ['Почему она так с ней поступает?'],
 ['Я хотел бы учиться в Бостоне.'],
 ['Том этим сейчас занимается.'],
 ['Том был просто не готов.'],
 ['Я ничего не хочу делать.'],
 ['Тому это тоже не нравится.'],
 ['Сколько сейчас времени в Париже?'],
 ['Тому было больше не к кому обратиться.'],
 ['Он сказал, что вчера был дома.'],
 ['Они никогда меня не слушают.'],
 ['Я поймал сегодня три рыбы.'],
 ['Я хочу что-нибудь сделать для Тома.'],
 ['Я сделал всё правильно?'],
 ['Я тоже завтра пойду в университет.'],
 ['Мне никогда не нужно было столько денег.'],
 ['Том не может жить без Мэри.'],
 ['Почему они так с ним поступают?'],
 ['Что могло с ними случиться?'],
 ['Мы будем делать всё по-своему.'],
 ['Где был Том весь день?'],
 ['Я здесь часто ем.'],

### Сохранение результатов

In [31]:
save_groups.save_groups(groups, "../output/solution5-result.json")